<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Content-Based Personalization with LightGBM on Spark

This notebook provides a quick example of how to train a [LightGBM](https://github.com/Microsoft/Lightgbm) model on Spark using [MMLSpark](https://github.com/Azure/mmlspark) for a content-based personalization scenario.

We use the [CRITEO dataset](https://www.kaggle.com/c/criteo-display-ad-challenge), a well known dataset of website ads that can be used to optimize the Click-Through Rate (CTR). The dataset consists of a series of numerical and categorical features and a binary label indicating whether the add has been clicked or not.

The model is based on [LightGBM](https://github.com/Microsoft/Lightgbm), which is a gradient boosting framework that uses tree-based learning algorithms. Finally, we take advantage of
[MMLSpark](https://github.com/Azure/mmlspark) library, which allows LightGBM to be called in a Spark environment and be computed distributely.

This scenario is a good example of **implicit feedback**, where binary labels indicate the interaction between a user and an item. This contrasts with explicit feedback, where the user explicitely rate the content, for example from 1 to 5. 


## Global Settings and Imports

This notebook can be run in a Spark environment in a DSVM or in Azure Databricks. For more details about the installation process, please refer to the [setup instructions](../../SETUP.md).

**NOTE for Azure Databricks:**
* A python script is provided to simplify setting up Azure Databricks with the correct dependencies. Run ```python tools/databricks_install.py -h``` for more details.
* MMLSpark should not be run on a cluster with autoscaling enabled. Disable the flag in the Azure Databricks Cluster configuration before running this notebook.

In [1]:
import os
import sys


import pyspark
from pyspark.ml import PipelineModel
from pyspark.ml.feature import FeatureHasher
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import papermill as pm
import scrapbook as sb


from recommenders.utils.notebook_utils import is_databricks
from recommenders.utils.spark_utils import start_or_get_spark
from recommenders.datasets.criteo import load_spark_df
from recommenders.datasets.spark_splitters import spark_random_split

# Setup MML Spark
from recommenders.utils.spark_utils import MMLSPARK_REPO, MMLSPARK_PACKAGE

# On Spark >3.0.0,<3.2.0, the following should be set:
#     MMLSPARK_PACKAGE = "com.microsoft.azure:synapseml_2.12:0.9.4"
packages = [MMLSPARK_PACKAGE]
repos = [MMLSPARK_REPO]
spark = start_or_get_spark(packages=packages, repositories=repos)
dbutils = None
print("MMLSpark version: {}".format(MMLSPARK_PACKAGE))

from synapse.ml.train import ComputeModelStatistics
from synapse.ml.lightgbm import LightGBMClassifier

print("System version: {}".format(sys.version))
print("PySpark version: {}".format(pyspark.version.__version__))

https://mmlspark.azureedge.net/maven added as a remote repository with the name: repo-1
Ivy Default Cache set to: /Users/p/.ivy2/cache
The jars for the packages stored in: /Users/p/.ivy2/jars
com.microsoft.azure#synapseml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-19fe0a9d-e1d1-4530-a5b2-a9a0cca960c9;1.0
	confs: [default]


:: loading settings :: url = jar:file:/Users/p/miniconda3/envs/alsrec/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found com.microsoft.azure#synapseml_2.12;0.9.5 in central
	found com.microsoft.azure#synapseml-core_2.12;0.9.5 in central
	found org.scalactic#scalactic_2.12;3.0.5 in central
	found org.scala-lang#scala-reflect;2.12.4 in central
	found io.spray#spray-json_2.12;1.3.2 in central
	found com.jcraft#jsch;0.1.54 in central
	found org.apache.httpcomponents#httpclient;4.5.6 in central
	found org.apache.httpcomponents#httpcore;4.4.10 in central
	found commons-logging#commons-logging;1.2 in central
	found commons-codec#commons-codec;1.10 in central
	found org.apache.httpcomponents#httpmime;4.5.6 in central
	found com.linkedin.isolation-forest#isolation-forest_3.2.0_2.12;2.0.8 in central
	found com.chuusai#shapeless_2.12;2.3.2 in central
	found org.typelevel#macro-compat_2.12;1.1.1 in central
	found org.apache.spark#spark-avro_2.12;3.2.0 in central
	found org.tukaani#xz;1.8 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.testng#testng;6.8.8 in central
	found org.bean

23/04/23 14:52:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


MMLSpark version: com.microsoft.azure:synapseml_2.12:0.9.5
System version: 3.9.16 (main, Mar  8 2023, 04:29:24) 
[Clang 14.0.6 ]
PySpark version: 3.4.0


In [2]:
# Criteo data size, it can be "sample" or "full"
DATA_SIZE = "sample"

# LightGBM parameters
# More details on parameters: https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html
NUM_LEAVES = 32
NUM_ITERATIONS = 50
LEARNING_RATE = 0.1
FEATURE_FRACTION = 0.8

# Model name
MODEL_NAME = 'lightgbm_criteo.mml'

## Data Preparation

The [Criteo Display Advertising Challenge](https://www.kaggle.com/c/criteo-display-ad-challenge) (Criteo DAC) dataset is a well-known industry benchmarking dataset for developing CTR prediction models, and is used frequently by research papers. The original dataset contains over 45M rows, but there is also a down-sampled dataset which has 100,000 rows (this can be used by setting `DATA_SIZE = "sample"`). Each row corresponds to a display ad served by Criteo and the first column is indicates whether this ad has been clicked or not.<br><br>
The dataset contains 1 label column and 39 feature columns, where 13 columns are integer values (int00-int12) and 26 columns are categorical features (cat00-cat25).<br><br>
What the columns represent is not provided, but for this case we can consider the integer and categorical values as features representing the user and / or item content. The label is binary and is an example of implicit feedback indicating a user's interaction with an item. With this dataset we can demonstrate how to build a model that predicts the probability of a user interacting with an item based on available user and item content features.


In [3]:
raw_data = load_spark_df(size=DATA_SIZE, spark=spark, dbutils=dbutils)
# visualize data
raw_data.limit(2).toPandas().head()

100%|█████████████████████████████████████| 8.58k/8.58k [00:01<00:00, 4.67kKB/s]
23/04/23 14:52:25 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,label,int00,int01,int02,int03,int04,int05,int06,int07,int08,...,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25
0,0,1,1,5,0,1382,4,15,2,181,...,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,None,3a171ecb,c5c50484,e8b83407,9727dd16
1,0,2,0,44,1,102,8,2,2,4,...,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,None,3a171ecb,43f13e8b,e8b83407,731c3655


### Feature Processing
The feature data provided has many missing values across both integer and categorical feature fields. In addition the categorical features have many distinct values, so effectively cleaning and representing the feature data is an important step prior to training a model.<br><br>
One of the simplest ways of managing both features that have missing values as well as high cardinality is to use the hashing trick. The [FeatureHasher](http://spark.apache.org/docs/latest/ml-features.html#featurehasher) transformer will pass integer values through and will hash categorical features into a sparse vector of lower dimensionality, which can be used effectively by LightGBM.<br><br>
First, the dataset is splitted randomly for training and testing and feature processing is applied to each dataset.

In [4]:
raw_data.limit(3).toPandas().head(3)

,label,int00,int01,int02,int03,int04,int05,int06,int07,int08,...,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25
0,0,1,1,5,0,1382,4,15,2,181,...,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,None,3a171ecb,c5c50484,e8b83407,9727dd16
1,0,2,0,44,1,102,8,2,2,4,...,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,None,3a171ecb,43f13e8b,e8b83407,731c3655
2,0,2,0,1,14,767,89,4,2,245,...,8efede7f,3412118d,None,None,e587c466,ad3062eb,3a171ecb,3b183c5c,None,None


In [13]:
raw_data.head(1)

[Row(label=0, int00=1, int01=1, int02=5, int03=0, int04=1382, int05=4, int06=15, int07=2, int08=181, int09=1, int10=2, int11=None, int12=2, cat00='68fd1e64', cat01='80e26c9b', cat02='fb936136', cat03='7b4723c4', cat04='25c83c98', cat05='7e0ccccf', cat06='de7995b8', cat07='1f89b562', cat08='a73ee510', cat09='a8cd5504', cat10='b2cb9c98', cat11='37c9c164', cat12='2824a5f6', cat13='1adce6ef', cat14='8ba8b39a', cat15='891b62e7', cat16='e5ba7672', cat17='f54016b9', cat18='21ddcdc9', cat19='b1252a9d', cat20='07b5194c', cat21=None, cat22='3a171ecb', cat23='c5c50484', cat24='e8b83407', cat25='9727dd16')]

In [5]:
raw_train, raw_test = spark_random_split(raw_data, ratio=0.8, seed=42)

In [6]:
columns = [c for c in raw_data.columns if c != 'label']
feature_processor = FeatureHasher(inputCols=columns, outputCol='features')

In [17]:
train = feature_processor.transform(raw_train)
test = feature_processor.transform(raw_test)

In [14]:
train.head(1)

[Row(label=0, int00=None, int01=-2, int02=None, int03=11, int04=589318, int05=None, int06=0, int07=18, int08=3, int09=None, int10=0, int11=None, int12=13, cat00='05db9164', cat01='d833535f', cat02='b00d1501', cat03='d16679b9', cat04='25c83c98', cat05='7e0ccccf', cat06='863329da', cat07='062b5529', cat08='7cc72ec2', cat09='d47daa4c', cat10='a89c45cb', cat11='e0d76380', cat12='a4fafa5b', cat13='07d13a8f', cat14='943169c2', cat15='1203a270', cat16='d4bb7bd8', cat17='281769c2', cat18=None, cat19=None, cat20='73d06dde', cat21='c9d4222a', cat22='c7dc6720', cat23='aee52b6f', cat24=None, cat25=None, features=SparseVector(262144, {1562: 1.0, 9567: 1.0, 20842: 1.0, 25911: 1.0, 31484: 1.0, 38328: 1.0, 46747: 18.0, 51885: 1.0, 54551: 1.0, 55096: 1.0, 61856: 1.0, 66551: 0.0, 68372: 1.0, 84264: 3.0, 92291: 1.0, 110783: 1.0, 127003: 589318.0, 131569: 1.0, 134624: 1.0, 170681: 1.0, 202012: 1.0, 212186: 1.0, 226453: 11.0, 238710: 1.0, 243754: 1.0, 248706: 1.0, 248972: 13.0, 256378: 1.0, 258299: 0.0, 26

## Model Training
In MMLSpark, the LightGBM implementation for binary classification is invoked using the `LightGBMClassifier` class and specifying the objective as `"binary"`. In this instance, the occurrence of positive labels is quite low, so setting the `isUnbalance` flag to true helps account for this imbalance.<br><br>

### Hyper-parameters
Below are some of the key [hyper-parameters](https://github.com/Microsoft/LightGBM/blob/master/docs/Parameters-Tuning.rst) for training a LightGBM classifier on Spark:
- `numLeaves`: the number of leaves in each tree
- `numIterations`: the number of iterations to apply boosting
- `learningRate`: the learning rate for training across trees
- `featureFraction`: the fraction of features used for training a tree

In [8]:
lgbm = LightGBMClassifier(
    labelCol="label",
    featuresCol="features",
    objective="binary",
    isUnbalance=True,
    boostingType="gbdt",
    boostFromAverage=True,
    baggingSeed=42,
    numLeaves=NUM_LEAVES,
    numIterations=NUM_ITERATIONS,
    learningRate=LEARNING_RATE,
    featureFraction=FEATURE_FRACTION
)

In [16]:
lgbm = LightGBMClassifier()

In [11]:
lgbm

LightGBMClassifier_cab425d32f27

### Model Training and Evaluation

In [15]:
from pyspark.ml.linalg import DenseVector

# Convert sparse features to dense vector
train = train.rdd.map(lambda row: (row['label'], DenseVector(row['features'].toArray()))).toDF(['label', 'features'])

# Train the LightGBM model
model = lgbm.fit(train)

TypeError: 'JavaPackage' object is not callable

In [18]:
model = lgbm.fit(train)
print("Success!")

Py4JJavaError: An error occurred while calling o155.getParam.
: java.util.NoSuchElementException: Param catSmooth does not exist.
	at org.apache.spark.ml.param.Params.$anonfun$getParam$2(params.scala:705)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.ml.param.Params.getParam(params.scala:705)
	at org.apache.spark.ml.param.Params.getParam$(params.scala:703)
	at org.apache.spark.ml.PipelineStage.getParam(Pipeline.scala:41)
	at jdk.internal.reflect.GeneratedMethodAccessor56.invoke(Unknown Source)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)

In [ ]:
predictions = model.transform(test)

In [ ]:
evaluator = (
    ComputeModelStatistics()
    .setScoredLabelsCol("prediction")
    .setLabelCol("label")
    .setEvaluationMetric("AUC")
)

result = evaluator.transform(predictions)
auc = result.select("AUC").collect()[0][0]
result.show()

In [ ]:
# Record results with papermill for tests
sb.glue("auc", auc)

## Model Saving 
The full pipeline for operating on raw data including feature processing and model prediction can be saved and reloaded for use in another workflow.

In [ ]:
# save model
pipeline = PipelineModel(stages=[feature_processor, model])
pipeline.write().overwrite().save(MODEL_NAME)

In [ ]:
# cleanup spark instance
if not is_databricks():
    spark.stop()

## Additional Reading
\[1\] Guolin Ke, Qi Meng, Thomas Finley, Taifeng Wang, Wei Chen, Weidong Ma, Qiwei Ye, and Tie-Yan Liu. 2017. LightGBM: A highly efficient gradient boosting decision tree. In Advances in Neural Information Processing Systems. 3146–3154. https://papers.nips.cc/paper/6907-lightgbm-a-highly-efficient-gradient-boosting-decision-tree.pdf <br>
\[2\] MML Spark: https://mmlspark.blob.core.windows.net/website/index.html <br>
